### Migratory Performance Assessment
#### Question: Is the migratory performance of rehab birds impacted by collisions?
- Compare with non-rehab birds
    - How many rehab versus non-rehab birds are there? *(4 rehab, 1 non)* 
    - " " detections for each? *(112 for rehab, 1303 for non)*
        - Detected rehabs: How many were detected elsewhere (repeat tags)?
- Movement 
    - How many birds are contained within a particular area?
    - Map the detections by color (rehab versus not)
        - Receivers:
            - Which receivers picked up birds?
            - Where are those receivers located?

Need to determine which tags were picked up twice (to compare movement trends)

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

In [7]:
# import using pandas
# Note - original csv has weird Mac Roman encoding.
df_detections = pd.read_csv('/Users/Monroe/Desktop/WoodcockDetction_data.csv', encoding = "mac_roman")

In [15]:
# view length and head
print(len(df_detections))
df_detections.head()

17989


,Unnamed: 0,motusTagID,tagDeployLon,tagDeployLat,speciesEN.x,recvDeployName,ts.h,recvLat,recvLon,tag,rehab,rehab.intake.date,condition,age,sex,speciesEN.y,release.site
0,1,28414,-80.0861,42.1602,White-throated Sparrow,Long_Point_Tip,2018-05-02 01:53:09,42.55,-80.05,234,N,NaN,NaN,ASY`,M,WTSP,Frys
1,2,28451,-81.9377,41.4861,American Woodcock,Girard,2018-05-02 03:23:23,42.00,-80.35,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
2,3,28451,-81.9377,41.4861,American Woodcock,Girard,2018-05-02 03:30:42,42.00,-80.35,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
3,4,28451,-81.9377,41.4861,American Woodcock,Girard,2018-05-02 03:31:32,42.00,-80.35,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
4,5,28451,-81.9377,41.4861,American Woodcock,PresqueIsle,2018-05-02 03:40:40,42.10,-80.15,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC


In [18]:
# filter for only Woodcock
df_woodcock = df_detections.loc[df_detections['speciesEN.x'] == "American Woodcock"]

# view length and head
print(len(df_woodcock)) #1415 detections total
df_woodcock.head()

1415


,Unnamed: 0,motusTagID,tagDeployLon,tagDeployLat,speciesEN.x,recvDeployName,ts.h,recvLat,recvLon,tag,rehab,rehab.intake.date,condition,age,sex,speciesEN.y,release.site
1,2,28451,-81.9377,41.4861,American Woodcock,Girard,2018-05-02 03:23:23,42.0,-80.35,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
2,3,28451,-81.9377,41.4861,American Woodcock,Girard,2018-05-02 03:30:42,42.0,-80.35,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
3,4,28451,-81.9377,41.4861,American Woodcock,Girard,2018-05-02 03:31:32,42.0,-80.35,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
4,5,28451,-81.9377,41.4861,American Woodcock,PresqueIsle,2018-05-02 03:40:40,42.1,-80.15,255,Y,28-Apr,mild,ASY,F,AMWO,LENSC
147,148,28449,-81.9377,41.4861,American Woodcock,PARC Honey Hut,2018-05-11 22:33:53,41.5,-81.95,254,Y,2-May,severe,SY,M,AMWO,LENSC


In [32]:
# number of rehab vs non detections
print(len(df_woodcock.loc[df_woodcock['rehab'] == "Y"]))
print(len(df_woodcock.loc[df_woodcock['rehab'] == "N"]))


# make a rehab and non-rehab df
df_rehab = df_woodcock.loc[df_woodcock['rehab'] == "Y"]
df_non = df_woodcock.loc[df_woodcock['rehab'] == "N"]

df_non.head()

112
1303


,Unnamed: 0,motusTagID,tagDeployLon,tagDeployLat,speciesEN.x,recvDeployName,ts.h,recvLat,recvLon,tag,rehab,rehab.intake.date,condition,age,sex,speciesEN.y,release.site
1283,1284,28445,-79.2674,40.1635,American Woodcock,PARC Banding,2018-05-24 13:49:05,40.15,-79.25,253,N,NaN,NaN,SY,F,AMWO,PARC
1284,1285,28445,-79.2674,40.1635,American Woodcock,PARC Banding,2018-05-24 13:58:03,40.15,-79.25,253,N,NaN,NaN,SY,F,AMWO,PARC
1285,1286,28445,-79.2674,40.1635,American Woodcock,PARC Banding,2018-05-24 14:05:12,40.15,-79.25,253,N,NaN,NaN,SY,F,AMWO,PARC
1286,1287,28445,-79.2674,40.1635,American Woodcock,PARC Banding,2018-05-24 14:13:50,40.15,-79.25,253,N,NaN,NaN,SY,F,AMWO,PARC
1287,1288,28445,-79.2674,40.1635,American Woodcock,PARC Banding,2018-05-24 14:24:30,40.15,-79.25,253,N,NaN,NaN,SY,F,AMWO,PARC


In [33]:
# check for repeat tags using pd.unqiue
print(len(df_rehab.motusTagID.unique()))

# check for number of unique non-rehab birds
print(len(df_non.motusTagID.unique()))

4
1


In [ ]:
# Need to determine which tags were picked up twice (to compare movement trends)